# NLP Unit 7

## Part 1: WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS
https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py

**See tutorial on Website -- Here we provide additional content / Exercise results**


In [72]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Exercise: Computing Word Embeddings: Continuous Bag-of-Words

In [73]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)
word_to_ix = {word: i for i, word in enumerate(vocab)}

def create_context_data(raw_text):

    data = []
    for i in range(2, len(raw_text) - 2):
        context = [raw_text[i - 2], raw_text[i - 1],
                   raw_text[i + 1], raw_text[i + 2]]
        target = raw_text[i]
        data.append((context, target))

    print(data[:5])
    print("\nNumber of training examples:", len(data))
    print("vocab_size", vocab_size)

    return data

data = create_context_data(raw_text)


[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]

Number of training examples: 58
vocab_size 49


In [74]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


make_context_vector(data[0][0], word_to_ix)  # example

tensor([22, 35, 46,  0])

**initial skeleton code:**

In [75]:
class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [76]:
EMBEDDING_DIM = 50
class CBOW(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size *  2 * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# create your model and train.  here are some functions to help you make
# the data ready for use by your module

In [77]:
losses = []
loss_function = nn.NLLLoss()
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(20):
    total_loss = 0
    for context, target in data:
                
        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in tensors)
        #print(context, target)
        context_idxs = make_context_vector(context, word_to_ix)  # example
        #print(context_ids)
        
        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_idxs)
        
        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a tensor)
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        # Get the Python number from a 1-element Tensor by calling tensor.item()
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[228.26647400856018, 225.08308219909668, 221.94002175331116, 218.83432626724243, 215.75911283493042, 212.70163893699646, 209.65577483177185, 206.61933946609497, 203.59706830978394, 200.58099508285522, 197.58383011817932, 194.58465337753296, 191.5872826576233, 188.6017553806305, 185.59955596923828, 182.58750987052917, 179.5686798095703, 176.5377436876297, 173.50407207012177, 170.46508395671844]


<font color="red">
    
### Do some predictions to see if works somehow :)

In [92]:
test_sentence = "We study the idea of computational processes".split()
test_data = create_context_data(test_sentence)
print(test_data)

## we need an idx-to-word
word_to_ix = {word: i for i, word in enumerate(vocab)}
ix_to_word = {i: word for word, i in word_to_ix.items()}
#print(ix_to_word)

with torch.no_grad():
    for context, target in test_data:
        context_idxs = make_context_vector(context, word_to_ix)  # example
        log_probs = model(context_idxs)
        
        # argmax give the index of the entry with the largest value
        pred_idx = log_probs.argmax().item()
        print(context, ix_to_word[pred_idx])
        

[(['We', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'computational'], 'idea'), (['the', 'idea', 'computational', 'processes'], 'of')]

Number of training examples: 3
vocab_size 49
[(['We', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'computational'], 'idea'), (['the', 'idea', 'computational', 'processes'], 'of')]
['We', 'study', 'idea', 'of'] the
['study', 'the', 'of', 'computational'] idea
['the', 'idea', 'computational', 'processes'] of


In [ ]:
### ok see it doesn't make too much sense yet, the training data is too small

# Part 4: LSTM: SEQUENCE MODELS AND LONG-SHORT TERM MEMORY NETWORKS

https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py


In [132]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6


{'The': 0, 'dog': 1, 'ate': 2, 'the': 3, 'apple': 4, 'Everybody': 5, 'read': 6, 'that': 7, 'book': 8}


In [133]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

In [134]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    print(inputs)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print("\n")
    print(tag_scores)


tensor([0, 1, 2, 3, 4])
tensor([[-0.8723, -1.0225, -1.5038],
        [-1.0115, -0.8982, -1.4740],
        [-1.0048, -0.9135, -1.4577],
        [-1.0608, -0.8628, -1.4617],
        [-1.0379, -0.8689, -1.4856]])


tensor([[-0.0400, -3.4591, -4.8612],
        [-4.4714, -0.0237, -4.4215],
        [-3.9015, -3.3954, -0.0552],
        [-0.0293, -3.8717, -4.8273],
        [-3.8551, -0.0268, -5.2412]])


### Exercise: Extend with Char-Level features!

In [205]:
EMBEDDING_DIM = 3 

# 0.) create a list of words in the training texts

sentences = [" ".join(sentence) for sentence, _ in training_data]

print(sentences)

# create the input and target sequences
inputs  = [s[:-1] for s in sentences]
targets = [s[1:] for s in sentences]

print(inputs)
print("Targets", targets)

# 1.) create char_to_ix
char_to_ix = {}
for sentence in sentences:
    for c in sentence:
        if c not in char_to_ix:
            char_to_ix[c] = len(char_to_ix)

print(char_to_ix)



vocab_size = len(char_to_ix)



['The dog ate the apple', 'Everybody read that book']
['The dog ate the appl', 'Everybody read that boo']
TT ['he dog ate the apple', 'verybody read that book']
{'T': 0, 'h': 1, 'e': 2, ' ': 3, 'd': 4, 'o': 5, 'g': 6, 'a': 7, 't': 8, 'p': 9, 'l': 10, 'E': 11, 'v': 12, 'r': 13, 'y': 14, 'b': 15, 'k': 16}


In [199]:


class CharLSTM(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size):
        super(CharLSTM, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2char = nn.Linear(hidden_dim, vocab_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        char_space = self.hidden2char(lstm_out.view(len(sentence), -1))
        char_scores = F.log_softmax(char_space, dim=1)
        return char_scores

In [217]:

model = CharLSTM(EMBEDDING_DIM, HIDDEN_DIM, len(char_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

print(inputs)
print("TTT", targets)

print(char_to_ix)

with torch.no_grad():
    inputs_in = prepare_sequence(inputs[0], char_to_ix)
    print(inputs_in)
    char_scores = model(inputs_in)

                 

for epoch in range(2000):  # again, normally you would NOT do 300 epochs, it is toy data
    for i in range(len(inputs)):
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sent_in = prepare_sequence(inputs[i], char_to_ix)
        targets_in = prepare_sequence(targets[i], char_to_ix)

        # Step 3. Run our forward pass.
        char_scores = model(sent_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(char_scores, targets_in)
        #print(loss)
        loss.backward()
        optimizer.step()
        
        
        if epoch%50 == 0:
        #print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
            print("Loss: {:.4f}".format(loss.item()))

# See what the scores are after training

with torch.no_grad():
    inputs_in = prepare_sequence(inputs[0], char_to_ix)
    print(inputs_in)
    char_scores = model(inputs_in)
    
    print("\n")
    print(char_scores)

['The dog ate the appl', 'Everybody read that boo']
TTT ['he dog ate the apple', 'verybody read that book']
{'T': 0, 'h': 1, 'e': 2, ' ': 3, 'd': 4, 'o': 5, 'g': 6, 'a': 7, 't': 8, 'p': 9, 'l': 10, 'E': 11, 'v': 12, 'r': 13, 'y': 14, 'b': 15, 'k': 16}
tensor([ 0,  1,  2,  3,  4,  5,  6,  3,  7,  8,  2,  3,  8,  1,  2,  3,  7,  9,
         9, 10])
Loss: 2.8382
Loss: 2.8191
Loss: 2.5086
Loss: 2.6656
Loss: 2.3774
Loss: 2.6231
Loss: 2.2739
Loss: 2.5900
Loss: 2.1625
Loss: 2.5534
Loss: 2.0468
Loss: 2.5058
Loss: 1.9312
Loss: 2.4368
Loss: 1.8171
Loss: 2.3421
Loss: 1.7058
Loss: 2.2243
Loss: 1.5912
Loss: 2.1121
Loss: 1.4786
Loss: 1.9888
Loss: 1.3745
Loss: 1.8660
Loss: 1.2780
Loss: 1.7283
Loss: 1.1876
Loss: 1.5861
Loss: 1.1058
Loss: 1.4577
Loss: 1.0688
Loss: 1.3474
Loss: 0.9882
Loss: 1.2550
Loss: 0.9232
Loss: 1.1666
Loss: 0.8634
Loss: 1.0823
Loss: 0.8153
Loss: 1.0111
Loss: 0.7570
Loss: 0.9469
Loss: 0.7006
Loss: 0.8691
Loss: 0.6645
Loss: 0.8182
Loss: 0.5999
Loss: 0.7626
Loss: 0.7554
Loss: 1.1590
L

In [218]:
### test
test_sent = "The dog "

print(char_to_ix['e'])

with torch.no_grad():
    inputs_in = prepare_sequence(test_sent, char_to_ix)
    print(inputs_in)
    scores = model(inputs_in)
    
    print("\n")
    print(scores)

2
tensor([0, 1, 2, 3, 4, 5, 6, 3])


tensor([[ -7.4438,  -0.7372,  -1.2152,  -6.4126,  -5.2044,  -1.9368,  -3.0482,
          -4.4317,  -6.4745,  -5.8520,  -7.4961,  -7.4767,  -5.2272,  -8.6973,
          -6.9849,  -8.6854,  -6.5108],
        [ -9.0105,  -4.3475,  -0.5458,  -8.2480,  -6.2382,  -0.9914,  -4.0227,
          -9.5098, -10.3575,  -6.8909,  -6.9912,  -9.2728,  -6.4027,  -9.7878,
          -5.7506, -10.9634,  -4.6732],
        [-10.6887,  -9.4974,  -7.4503,  -0.0337,  -5.3973,  -9.4610,  -4.6711,
         -10.8939, -11.1271, -10.9869,  -6.3833, -10.7422,  -9.1619,  -9.0728,
          -8.1038,  -8.5869,  -4.1341],
        [ -8.2791,  -9.3051,  -5.3962,  -4.5416,  -0.4460,  -6.5215,  -4.8627,
          -3.5401,  -4.0985,  -8.7080,  -7.2373,  -7.9702,  -8.3419,  -2.6256,
          -2.6396,  -1.9627,  -5.5828],
        [ -8.7698,  -2.0899,  -2.7384, -11.3802,  -4.0750,  -0.3209,  -3.0946,
          -3.9576,  -6.3019,  -6.8237, -10.7937,  -8.4396,  -7.3011,  -9.6433,
          -7.